In [ ]:
import sys
import numpy as np
import math

In [ ]:
def get_gyration_matrix(cluster):
    # Moment of gyration tensor.
    COMCluster = translate_to_com(cluster)
    # replace with np.sum is better, but idk how
    Ixx = 0
    Iyy = 0
    Izz = 0
    Ixy = 0
    Iyz = 0
    Ixz = 0
    for position in COMCluster:
      Ixx += position[0]**2
      Iyy += position[1]**2
      Izz += position[2]**2
      Ixy += position[0]*position[1]
      Iyz += position[1]*position[2]
      Ixz += position[0]*position[2]
    Ixx /= len(cluster)
    Iyy /= len(cluster)
    Izz /= len(cluster)
    Ixy /= len(cluster)
    Iyz /= len(cluster)
    Ixz /= len(cluster)
    I = np.array([[Ixx, Ixy, Ixz], [Ixy, Iyy, Iyz], [Ixz, Iyz, Izz]])
    return I

def get_principal_gyration(I):
    Ip = np.linalg.eigvals(I)
    Ip.sort()
    return [math.sqrt(Ip[0]), math.sqrt(Ip[1]), math.sqrt(Ip[2])]

def normalize(lambdas): #assume 3 element array!
    preFactor = 1 / (math.sqrt(lambdas[0]**2 + lambdas[1]**2 + lambdas[2]**2))
    return np.dot(preFactor, lambdas)

def asphericity(lambdas): #assume 3 element array is sorted
    return ((lambdas[2])**2 - 1/2 * ((lambdas[0])**2 + (lambdas[1])**2))

def acylindricity(lambdas):
    return (lambdas[1])**2 - lambdas[0]**2

def anisotropy(lambdas):
    return (3/2 * (lambdas[0]**4 + lambdas[1] ** 4 + lambdas[2] ** 4)/(lambdas[0]**2 + lambdas[1]**2 + lambdas[2]**2)**2 - 1/2)

def unwrap(cluster):

    coord1 = 0.0
    coord2 = 33.0
    boxLength = coord2 - coord1
    coordCenter = (coord1 + coord2)/2

    flipX = True #check if we need to flip x. if any point is within somoe distance of the middle of a dimension, dont flip 
    flipY = True
    flipZ = True

    for position in cluster:
      if abs(position[0] - coordCenter) < 1.12: #arbitrary cutoff
        flipX = False
      if abs(position[1] - coordCenter) < 1.12: #arbitrary cutoff
        flipY = False
      if abs(position[2] - coordCenter) < 1.12: #arbitrary cutoff
        flipZ = False

    for position in cluster:
      if (position[0] > coordCenter) and flipX:
        position[0] = position[0] - boxLength
      if (position[1] > coordCenter) and flipY:
        position[1] = position[1] - boxLength
      if (position[2] > coordCenter) and flipZ:
        position[2] = position[2] - boxLength

def translate_to_com(cluster):
    comX = 0
    comY = 0
    comZ = 0
    for position in cluster:
      comX += position[0]
      comY += position[1]
      comZ += position[2]
    comX /= len(cluster)
    comY /= len(cluster)
    comZ /= len(cluster)
    COMCluster = []
    for position in cluster:
        COMCluster.append([position[0] - comX, position[1] - comY, position[2] - comZ])
    return COMCluster

Import the .xyz of a single timestep, as well as unwrap.



In [ ]:
    f = input('Enter name of XYZ file:  ') 
    xyz = open(f)
    N = int(xyz.readline())
    header = xyz.readline()
    # define clusters
    O = []
    N = []
    C = []
    S = []
    P = []
    Z = []
    # atom_symbol, coords = ([] for i in range (2))
    for line in xyz:
        atom,x,y,z = line.split()
        if atom == 'O':
            O.append([float(x), float(y), float(z)])
        if atom == 'N':
            N.append([float(x), float(y), float(z)])
        if atom == 'C':
            C.append([float(x), float(y), float(z)])
        if atom == 'S':
            S.append([float(x), float(y), float(z)])
        if atom == 'P':
            P.append([float(x), float(y), float(z)])
        if atom == 'Z':
            Z.append([float(x), float(y), float(z)])
    xyz.close()

Enter name of XYZ file:  input.xyz


In [ ]:
output = open("output.txt", "w")

def process(cluster):
    unwrap(cluster)
    unNormalizedLambdas = get_principal_gyration(get_gyration_matrix(cluster))
    output.write("Unnormalized Lambdas: {} {} {}\n".format(unNormalizedLambdas[0], unNormalizedLambdas[1], unNormalizedLambdas[2]))
    normalizedLambdas = normalize(unNormalizedLambdas)
    output.write("Normalized Lambdas: {} {} {}\n".format(normalizedLambdas[0], normalizedLambdas[1], normalizedLambdas[2]))
    output.write("Asphericity: {}\n".format(asphericity(normalizedLambdas)))
    output.write("Acylindricity: {}\n".format(acylindricity(normalizedLambdas)))
    output.write("Relative Shape Anisotropy: {}\n".format(anisotropy(normalizedLambdas)))


output.write("--------- Cluster O ----------\n")
process(O)
output.write("--------- Cluster N ----------\n")
process(N)
output.write("--------- Cluster C ----------\n")
process(C)
output.write("--------- Cluster S ----------\n")
process(S)
output.write("--------- Cluster P ----------\n")
process(P)
output.write("--------- Cluster Z ----------\n")
process(Z)

output.close()